
## 1. Theory & Pseudocode

**DFS (Depth-First Search)** explores deep before backtracking. Useful for topological sorting, connected components, backtracking puzzles.



In [9]:
DFS(node):
  visit node
  for neighbor in node.neighbors:
    if not visited: DFS(neighbor)


SyntaxError: invalid syntax (1506184331.py, line 1)

**BFS (Breadth-First Search)** explores level by level, guaranteeing shortest path in unweighted graphs. Used in routing, web crawling.

In [ ]:
BFS(start):
  queue = [start]
  while queue:
    node = dequeue(queue)
    for neighbor in node.neighbors:
      if not visited: enqueue(queue, neighbor)

**IDDFS (Iterative Deepening DFS)** combines DFS’s space efficiency with BFS’s completeness up to a depth limit. Ideal when target depth is unknown but memory is tight.

In [ ]:
for limit in range(max_depth+1):
  depth_limited_DFS(start, limit)

**Bi-BFS (Bidirectional BFS)** runs two simultaneous BFS from start and target, meeting in the middle to reduce search space.

In [ ]:
while both queues not empty:
  expand one level forward
  expand one level backward
  if intersection: reconstruct path

## 2. Algorithm Implementations

In [1]:
def dfs_recursive(graph, start, visited=None, depth=0, stats=None):
    if visited is None:
        visited = set()
    if stats is None:
        stats = {'max_depth': 0, 'ops': 0}
    visited.add(start)
    stats['max_depth'] = max(stats['max_depth'], depth)
    for nbr in graph[start]:
        stats['ops'] += 1
        if nbr not in visited:
            dfs_recursive(graph, nbr, visited, depth+1, stats)
    return visited, stats

In [2]:
def dfs_iterative(graph, start):
    visited, ops = set(), 0
    stack = [(start, 0)]
    max_depth = 0
    while stack:
        node, depth = stack.pop()
        if node not in visited:
            visited.add(node)
            max_depth = max(max_depth, depth)
            for nbr in graph[node]:
                ops += 1
                if nbr not in visited:
                    stack.append((nbr, depth+1))
    return visited, {'max_depth': max_depth, 'ops': ops}

In [3]:
from collections import deque

def bfs(graph, start):
    visited = {start}
    queue = deque([start])
    frontier_sizes, ops = [], 0
    while queue:
        frontier_sizes.append(len(queue))
        for _ in range(len(queue)):
            node = queue.popleft()
            for nbr in graph[node]:
                ops += 1
                if nbr not in visited:
                    visited.add(nbr)
                    queue.append(nbr)
    return visited, {'max_frontier': max(frontier_sizes), 'ops': ops}

In [5]:
def iterative_deepening_dfs(graph, start, max_depth, target=None):
    stats = {'ops': 0, 'max_frontier': 0, 'steps': 0}
    for limit in range(max_depth+1):
        visited = set()
        stack = [(start, 0, [start])]
        current_max = 0
        while stack:
            current_max = max(current_max, len(stack))
            node, depth, path = stack.pop()
            stats['steps'] += 1
            if node not in visited:
                visited.add(node)
                if target is not None and node == target:
                    stats.update({'max_frontier': current_max, 'path': path, 'path_length': len(path)})
                    return visited, stats
                if depth < limit:
                    for nbr in reversed(list(graph[node])):
                        stats['ops'] += 1
                        if nbr not in visited:
                            stack.append((nbr, depth+1, path+[nbr]))
        stats['max_frontier'] = max(stats['max_frontier'], current_max)
        if len(visited) == len(graph):
            break
    stats['visited_count'] = len(visited)
    return visited, stats

In [6]:
def bidirectional_bfs(graph, start, target):
    if start == target:
        return {start}, {'path_length': 1, 'ops': 0, 'steps': 0, 'max_frontier': 1, 'visited_count': 1}
    forward_q, backward_q = deque([start]), deque([target])
    f_vis, b_vis = {start:[start]}, {target:[target]}
    ops = steps = 0
    max_frontier = 2
    while forward_q and backward_q:
        steps += 1
        max_frontier = max(max_frontier, len(forward_q) + len(backward_q))
        # expand forward
        for _ in range(len(forward_q)):
            u = forward_q.popleft()
            for nbr in graph[u]:
                ops += 1
                if nbr not in f_vis:
                    f_vis[nbr] = f_vis[u] + [nbr]
                    forward_q.append(nbr)
                    if nbr in b_vis:
                        path = f_vis[nbr] + b_vis[nbr][-2::-1]
                        visited = set(f_vis) | set(b_vis)
                        return visited, {'path': path, 'path_length': len(path), 'ops': ops, 'steps': steps, 'max_frontier': max_frontier, 'visited_count': len(visited)}
        # expand backward
        for _ in range(len(backward_q)):
            u = backward_q.popleft()
            for nbr in graph[u]:
                ops += 1
                if nbr not in b_vis:
                    b_vis[nbr] = b_vis[u] + [nbr]
                    backward_q.append(nbr)
                    if nbr in f_vis:
                        path = f_vis[nbr] + b_vis[nbr][-2::-1]
                        visited = set(f_vis) | set(b_vis)
                        return visited, {'path': path, 'path_length': len(path), 'ops': ops, 'steps': steps, 'max_frontier': max_frontier, 'visited_count': len(visited)}
    visited = set(f_vis) | set(b_vis)
    return visited, {'path_length': float('inf'), 'ops': ops, 'steps': steps, 'max_frontier': max_frontier, 'visited_count': len(visited)}

## 3. Test Graph Generators

In [10]:
import networkx as nx

def generate_sparse_graph(n): return nx.gnm_random_graph(n, n-1, seed=42)
def generate_dense_graph(n): return nx.gnm_random_graph(n, n*(n-1)//2, seed=42)
def generate_grid_graph(n):
    side = int(n**0.5)
    G = nx.grid_2d_graph(side, side)
    return nx.convert_node_labels_to_integers(G)
def generate_cycle_graph(n): return nx.cycle_graph(n)
def generate_star_graph(n): return nx.star_graph(n-1)
def generate_complete_graph(n): return nx.complete_graph(n)
def generate_barabasi_graph(n): return nx.barabasi_albert_graph(n, max(1, n//10), seed=42)
def generate_ws_graph(n, p): return nx.watts_strogatz_graph(n, 4, p, seed=42)
def generate_geo_graph(n, r): return nx.random_geometric_graph(n, r, seed=42)

graph_types = {
    'Sparse': generate_sparse_graph,
    'Dense': generate_dense_graph,
    'Grid': generate_grid_graph,
    'Cycle': generate_cycle_graph,
    'Star': generate_star_graph,
    'Complete': generate_complete_graph,
    'Barabasi': generate_barabasi_graph,
    'Watts-Strogatz': lambda n: generate_ws_graph(n, 0.3),
    'Random-Geometric': lambda n: generate_geo_graph(n, 0.1)
}

## 4. Metrics & Measurement Functions

In [11]:
def measure(func, *args, repeats=5):
    times, mems, stats_list = [], [], []
    for _ in range(repeats):
        tracemalloc.start()
        t0 = time.perf_counter()
        result, stats = func(*args)
        elapsed = time.perf_counter() - t0
        peak = tracemalloc.get_traced_memory()[1] / 1024
        tracemalloc.stop()
        times.append(elapsed)
        mems.append(peak)
        stats_list.append(stats)
    summary = {
        'time_mean': statistics.mean(times), 'time_std': statistics.stdev(times),
        'mem_mean': statistics.mean(mems), 'mem_std': statistics.stdev(mems),
        'stats': stats_list
    }
    return summary


## 5. Empirical Experiments

In [12]:

import pandas as pd

sizes = [100, 200, 300]
records = []
for name, gen in graph_types.items():
    for n in sizes:
        G = gen(n)
        graph = nx.to_dict_of_sets(G)
        d_rec = measure(dfs_recursive, graph, 0)
        d_itr = measure(dfs_iterative, graph, 0)
        bfs_m = measure(bfs, graph, 0)
        iddfs_m = measure(iterative_deepening_dfs, graph, 0, 10)
        bbfs_m = measure(bidirectional_bfs, graph, 0, n-1)
        records.append({
            'Graph': name, 'Size': n,
            'DFS Rec Time': d_rec['time_mean'], 'DFS Rec Time Std': d_rec['time_std'],
            'DFS Rec Mem': d_rec['mem_mean'], 'DFS Rec Mem Std': d_rec['mem_std'],
            'DFS Itr Time': d_itr['time_mean'], 'DFS Itr Mem': d_itr['mem_mean'],
            'BFS Time': bfs_m['time_mean'], 'BFS Mem': bfs_m['mem_mean'],
            'IDDFS Time': iddfs_m['time_mean'], 'IDDFS Mem': iddfs_m['mem_mean'],
            'Bi-BFS Time': bbfs_m['time_mean'], 'Bi-BFS Mem': bbfs_m['mem_mean']
        })
df = pd.DataFrame(records)
df

AttributeError: module 'networkx' has no attribute 'to_dict_of_sets'

## 6. Visualization

In [13]:
import matplotlib.pyplot as plt

def plot_metric(metric, ylabel):
    plt.figure(figsize=(12,6))
    for alg in ['DFS Rec', 'DFS Itr', 'BFS', 'IDDFS', 'Bi-BFS']:
        mean_col = f"{alg} {metric}"
        std_col = f"{alg} {metric} Std" if metric=='Time' else None
        for name in df['Graph'].unique():
            sub = df[df['Graph']==name]
            if std_col and std_col in sub:
                plt.errorbar(sub['Size'], sub[mean_col], yerr=sub[std_col], capsize=3,
                             label=f"{alg} - {name}")
            else:
                plt.plot(sub['Size'], sub[mean_col], label=f"{alg} - {name}")
    plt.xlabel('Nodes')
    plt.ylabel(ylabel)
    plt.title(f'{metric} Comparison')
    plt.legend(); plt.grid(True)

plot_metric('Time', 'Seconds')
plot_metric('Mem', 'KB')

NameError: name 'df' is not defined

<Figure size 1200x600 with 0 Axes>